Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [6]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f,encoding='latin1')
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [7]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale or color

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [8]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [9]:
batch_size = 16
patch_size = 5
# in_channels (num_channels) = 1
# out_channels = depth
depth = 16
num_hidden = 64
strides = 2

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, strides, strides, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, strides, strides, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.605437
Minibatch accuracy: 6.2%
Validation accuracy: 12.2%
Minibatch loss at step 50: 1.498927
Minibatch accuracy: 31.2%
Validation accuracy: 51.7%
Minibatch loss at step 100: 1.001370
Minibatch accuracy: 62.5%
Validation accuracy: 71.2%
Minibatch loss at step 150: 0.330139
Minibatch accuracy: 93.8%
Validation accuracy: 73.8%
Minibatch loss at step 200: 0.720368
Minibatch accuracy: 87.5%
Validation accuracy: 77.4%
Minibatch loss at step 250: 1.224110
Minibatch accuracy: 62.5%
Validation accuracy: 78.1%
Minibatch loss at step 300: 0.380268
Minibatch accuracy: 93.8%
Validation accuracy: 79.7%
Minibatch loss at step 350: 0.491906
Minibatch accuracy: 93.8%
Validation accuracy: 76.9%
Minibatch loss at step 400: 0.280962
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.916006
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 500: 0.747534
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [11]:
def get_shape(data):
    shape = data.get_shape().as_list()
    print(shape[0], shape[1], shape[2], shape[3])

In [106]:
batch_size = 16
patch_size = 5
# in_channels (num_channels) = 1
# out_channels = depth
depth = 16
num_hidden = 64
strides = 2
ksize = 2
batch_size_2 = test_dataset.shape[0] // 10

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(
    tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.placeholder(
    tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))
  tf_test_dataset = tf.placeholder(
    tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model_max_pool(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    
    
    hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
    
    pool_1 = tf.nn.max_pool(hidden_1, [1, ksize, ksize, 1], [1, strides, strides, 1], padding='SAME')
    
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    
    hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
    
    pool_2 = tf.nn.max_pool(hidden_2, [1, ksize, ksize, 1], [1, strides, strides, 1], padding='SAME')
    
    
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

  # Training computation.
  logits = model_max_pool(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model_max_pool(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model_max_pool(tf_test_dataset))
  

In [96]:
print(test_prediction)

Tensor("Softmax_2:0", shape=(1000, 10), dtype=float32)


In [68]:
def batch(data,data_labels,batch_size):
    offset = (step * batch_size) % (data_labels.shape[0] - batch_size)
    batch_data = data[offset:(offset + batch_size), :, :, :]
    batch_labels = data_labels[offset:(offset + batch_size), :]
    return batch_data, batch_labels

In [107]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    batch_data, batch_labels = batch(train_dataset, train_labels,batch_size)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      a = []  
      for step in range(10):
        valid_batch_data, valid_batch_labels = batch(valid_dataset, valid_labels, batch_size_2)
        feed_dict = {tf_valid_dataset : valid_batch_data}
        valid_predictions= session.run(valid_prediction, feed_dict=feed_dict)
        a.append(accuracy(valid_predictions, valid_batch_labels))
      print('Valid accuracy: %.1f%%' % (sum(a)/len(a)))
  a = []  
  for step in range(10):
    test_batch_data, test_batch_labels = batch(test_dataset, test_labels, batch_size_2)
    feed_dict = {tf_test_dataset : test_batch_data}
    test_predictions= session.run(test_prediction, feed_dict=feed_dict)
    a.append(accuracy(test_predictions, test_batch_labels))
  print('Test accuracy: %.1f%%' % (sum(a)/len(a)))

Initialized
Minibatch loss at step 0: 4.243682
Minibatch accuracy: 0.0%
Valid accuracy: 10.0%
Minibatch loss at step 50: 1.742559
Minibatch accuracy: 37.5%
Valid accuracy: 48.2%
Minibatch loss at step 100: 1.045639
Minibatch accuracy: 62.5%
Valid accuracy: 54.1%
Minibatch loss at step 150: 0.465825
Minibatch accuracy: 87.5%
Valid accuracy: 73.4%
Minibatch loss at step 200: 0.878713
Minibatch accuracy: 75.0%
Valid accuracy: 79.0%
Minibatch loss at step 250: 1.395222
Minibatch accuracy: 62.5%
Valid accuracy: 78.2%
Minibatch loss at step 300: 0.288110
Minibatch accuracy: 93.8%
Valid accuracy: 80.9%
Minibatch loss at step 350: 0.414942
Minibatch accuracy: 93.8%
Valid accuracy: 78.6%
Minibatch loss at step 400: 0.295715
Minibatch accuracy: 87.5%
Valid accuracy: 81.7%
Minibatch loss at step 450: 0.801758
Minibatch accuracy: 81.2%
Valid accuracy: 81.1%
Minibatch loss at step 500: 0.666861
Minibatch accuracy: 87.5%
Valid accuracy: 82.2%
Minibatch loss at step 550: 1.011748
Minibatch accuracy: 

**Performance Increase**

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

### add learning rate decay

In [109]:
batch_size = 16
patch_size = 5
# in_channels (num_channels) = 1
# out_channels = depth
depth = 16
num_hidden = 64
strides = 2
ksize = 2
dropout_rate = 0.5
batch_size_2 = test_dataset.shape[0] // 10

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.placeholder(
    tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))
  tf_test_dataset = tf.placeholder(
    tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))

  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  
  # Training computation.
  logits = model_max_pool(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 100000, 0.9)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model_max_pool(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model_max_pool(tf_test_dataset))

In [110]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    batch_data, batch_labels = batch(train_dataset, train_labels,batch_size)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      a = []  
      for step in range(10):
        valid_batch_data, valid_batch_labels = batch(valid_dataset, valid_labels, batch_size_2)
        feed_dict = {tf_valid_dataset : valid_batch_data}
        valid_predictions= session.run(valid_prediction, feed_dict=feed_dict)
        a.append(accuracy(valid_predictions, valid_batch_labels))
      print('Valid accuracy: %.1f%%' % (sum(a)/len(a)))
  a = []  
  for step in range(10):
    test_batch_data, test_batch_labels = batch(test_dataset, test_labels, batch_size_2)
    feed_dict = {tf_test_dataset : test_batch_data}
    test_predictions= session.run(test_prediction, feed_dict=feed_dict)
    a.append(accuracy(test_predictions, test_batch_labels))
  print('Test accuracy: %.1f%%' % (sum(a)/len(a)))

Initialized
Minibatch loss at step 0: 2.802702
Minibatch accuracy: 18.8%
Valid accuracy: 10.2%
Minibatch loss at step 50: 2.218733
Minibatch accuracy: 25.0%
Valid accuracy: 23.2%
Minibatch loss at step 100: 1.599844
Minibatch accuracy: 31.2%
Valid accuracy: 48.8%
Minibatch loss at step 150: 0.800932
Minibatch accuracy: 81.2%
Valid accuracy: 70.6%
Minibatch loss at step 200: 0.972942
Minibatch accuracy: 75.0%
Valid accuracy: 77.3%
Minibatch loss at step 250: 1.249375
Minibatch accuracy: 56.2%
Valid accuracy: 76.9%
Minibatch loss at step 300: 0.341971
Minibatch accuracy: 87.5%
Valid accuracy: 79.2%
Minibatch loss at step 350: 0.538116
Minibatch accuracy: 93.8%
Valid accuracy: 79.1%
Minibatch loss at step 400: 0.189741
Minibatch accuracy: 100.0%
Valid accuracy: 80.5%
Minibatch loss at step 450: 0.708903
Minibatch accuracy: 87.5%
Valid accuracy: 79.0%
Minibatch loss at step 500: 0.534332
Minibatch accuracy: 87.5%
Valid accuracy: 81.1%
Minibatch loss at step 550: 0.958018
Minibatch accuracy

**Add learning rate decay decrease performance**

### Add dropout to each hidden layer

In [111]:
def model_max_pool_dropout(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')    
    hidden_1 = tf.nn.dropout(tf.nn.relu(conv_1 + layer1_biases),dropout_rate)
    pool_1 = tf.nn.max_pool(hidden_1, [1, ksize, ksize, 1], [1, strides, strides, 1], padding='SAME')
    
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden_2 = tf.nn.dropout(tf.nn.relu(conv_2 + layer2_biases),dropout_rate)
    pool_2 = tf.nn.max_pool(hidden_2, [1, ksize, ksize, 1], [1, strides, strides, 1], padding='SAME')
    
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),dropout_rate)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

In [112]:
batch_size = 16
patch_size = 5
# in_channels (num_channels) = 1
# out_channels = depth
depth = 16
num_hidden = 64
strides = 2
ksize = 2
dropout_rate = 0.5
batch_size_2 = test_dataset.shape[0] // 10

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.placeholder(
    tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))
  tf_test_dataset = tf.placeholder(
    tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))

  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  
  # Training computation.
  logits = model_max_pool_dropout(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model_max_pool_dropout(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model_max_pool_dropout(tf_test_dataset))

In [113]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    batch_data, batch_labels = batch(train_dataset, train_labels,batch_size)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      a = []  
      for step in range(10):
        valid_batch_data, valid_batch_labels = batch(valid_dataset, valid_labels, batch_size_2)
        feed_dict = {tf_valid_dataset : valid_batch_data}
        valid_predictions= session.run(valid_prediction, feed_dict=feed_dict)
        a.append(accuracy(valid_predictions, valid_batch_labels))
      print('Valid accuracy: %.1f%%' % (sum(a)/len(a)))
  a = []  
  for step in range(10):
    test_batch_data, test_batch_labels = batch(test_dataset, test_labels, batch_size_2)
    feed_dict = {tf_test_dataset : test_batch_data}
    test_predictions= session.run(test_prediction, feed_dict=feed_dict)
    a.append(accuracy(test_predictions, test_batch_labels))
  print('Test accuracy: %.1f%%' % (sum(a)/len(a)))

Initialized
Minibatch loss at step 0: 8.138836
Minibatch accuracy: 6.2%
Valid accuracy: 9.1%
Minibatch loss at step 50: 1.732248
Minibatch accuracy: 56.2%
Valid accuracy: 20.8%
Minibatch loss at step 100: 2.063099
Minibatch accuracy: 18.8%
Valid accuracy: 36.2%
Minibatch loss at step 150: 2.592981
Minibatch accuracy: 37.5%
Valid accuracy: 42.3%
Minibatch loss at step 200: 1.202296
Minibatch accuracy: 50.0%
Valid accuracy: 52.4%
Minibatch loss at step 250: 1.713544
Minibatch accuracy: 43.8%
Valid accuracy: 56.2%
Minibatch loss at step 300: 0.784932
Minibatch accuracy: 81.2%
Valid accuracy: 61.6%
Minibatch loss at step 350: 1.260552
Minibatch accuracy: 68.8%
Valid accuracy: 59.5%
Minibatch loss at step 400: 0.811566
Minibatch accuracy: 75.0%
Valid accuracy: 63.1%
Minibatch loss at step 450: 1.565977
Minibatch accuracy: 62.5%
Valid accuracy: 66.5%
Minibatch loss at step 500: 0.981295
Minibatch accuracy: 68.8%
Valid accuracy: 66.4%
Minibatch loss at step 550: 0.979581
Minibatch accuracy: 7

**Add dropout to each hidden layer decrease performance**

### Add dropout to final hidden layer

In [114]:
def model_max_pool_dropout(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')    
    hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
    pool_1 = tf.nn.max_pool(hidden_1, [1, ksize, ksize, 1], [1, strides, strides, 1], padding='SAME')
    
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
    pool_2 = tf.nn.max_pool(hidden_2, [1, ksize, ksize, 1], [1, strides, strides, 1], padding='SAME')
    
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),dropout_rate)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

In [115]:
batch_size = 16
patch_size = 5
# in_channels (num_channels) = 1
# out_channels = depth
depth = 16
num_hidden = 64
strides = 2
ksize = 2
dropout_rate = 0.5
batch_size_2 = test_dataset.shape[0] // 10


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.placeholder(
    tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))
  tf_test_dataset = tf.placeholder(
    tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))


  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  
  # Training computation.
  logits = model_max_pool_dropout(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model_max_pool_dropout(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model_max_pool_dropout(tf_test_dataset))

In [116]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    batch_data, batch_labels = batch(train_dataset, train_labels,batch_size)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      a = []  
      for step in range(10):
        valid_batch_data, valid_batch_labels = batch(valid_dataset, valid_labels, batch_size_2)
        feed_dict = {tf_valid_dataset : valid_batch_data}
        valid_predictions= session.run(valid_prediction, feed_dict=feed_dict)
        a.append(accuracy(valid_predictions, valid_batch_labels))
      print('Valid accuracy: %.1f%%' % (sum(a)/len(a)))
  a = []  
  for step in range(10):
    test_batch_data, test_batch_labels = batch(test_dataset, test_labels, batch_size_2)
    feed_dict = {tf_test_dataset : test_batch_data}
    test_predictions= session.run(test_prediction, feed_dict=feed_dict)
    a.append(accuracy(test_predictions, test_batch_labels))
  print('Test accuracy: %.1f%%' % (sum(a)/len(a)))

Initialized
Minibatch loss at step 0: 5.407548
Minibatch accuracy: 12.5%
Valid accuracy: 10.1%
Minibatch loss at step 50: 2.298061
Minibatch accuracy: 6.2%
Valid accuracy: 13.2%
Minibatch loss at step 100: 2.205603
Minibatch accuracy: 37.5%
Valid accuracy: 18.9%
Minibatch loss at step 150: 1.669419
Minibatch accuracy: 43.8%
Valid accuracy: 27.0%
Minibatch loss at step 200: 1.850711
Minibatch accuracy: 37.5%
Valid accuracy: 39.3%
Minibatch loss at step 250: 1.930590
Minibatch accuracy: 31.2%
Valid accuracy: 47.9%
Minibatch loss at step 300: 1.012352
Minibatch accuracy: 68.8%
Valid accuracy: 59.1%
Minibatch loss at step 350: 0.817988
Minibatch accuracy: 75.0%
Valid accuracy: 60.2%
Minibatch loss at step 400: 0.830972
Minibatch accuracy: 68.8%
Valid accuracy: 64.2%
Minibatch loss at step 450: 1.269055
Minibatch accuracy: 56.2%
Valid accuracy: 67.7%
Minibatch loss at step 500: 1.033329
Minibatch accuracy: 75.0%
Valid accuracy: 67.2%
Minibatch loss at step 550: 0.561945
Minibatch accuracy: 

**Add dropout to final hidden layer is better than add dropout to all layers, but it decrease performance compared to just use CNN+max pool**

### Try avg_pool

In [117]:
def model_avg_pool_dropout(data):
    conv_1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')    
    hidden_1 = tf.nn.relu(conv_1 + layer1_biases)
    pool_1 = tf.nn.avg_pool(hidden_1, [1, ksize, ksize, 1], [1, strides, strides, 1], padding='SAME')
    
    conv_2 = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden_2 = tf.nn.relu(conv_2 + layer2_biases)
    pool_2 = tf.nn.max_pool(hidden_2, [1, ksize, ksize, 1], [1, strides, strides, 1], padding='SAME')
    
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

In [118]:
batch_size = 16
patch_size = 5
# in_channels (num_channels) = 1
# out_channels = depth
depth = 16
num_hidden = 64
strides = 2
ksize = 2
dropout_rate = 0.5
batch_size_2 = test_dataset.shape[0] // 10


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.placeholder(
    tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))
  tf_test_dataset = tf.placeholder(
    tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))

  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  
  # Training computation.
  logits = model_avg_pool_dropout(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model_avg_pool_dropout(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model_avg_pool_dropout(tf_test_dataset))

In [119]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    batch_data, batch_labels = batch(train_dataset, train_labels,batch_size)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      a = []  
      for step in range(10):
        valid_batch_data, valid_batch_labels = batch(valid_dataset, valid_labels, batch_size_2)
        feed_dict = {tf_valid_dataset : valid_batch_data}
        valid_predictions= session.run(valid_prediction, feed_dict=feed_dict)
        a.append(accuracy(valid_predictions, valid_batch_labels))
      print('Valid accuracy: %.1f%%' % (sum(a)/len(a)))
  a = []  
  for step in range(10):
    test_batch_data, test_batch_labels = batch(test_dataset, test_labels, batch_size_2)
    feed_dict = {tf_test_dataset : test_batch_data}
    test_predictions= session.run(test_prediction, feed_dict=feed_dict)
    a.append(accuracy(test_predictions, test_batch_labels))
  print('Test accuracy: %.1f%%' % (sum(a)/len(a)))

Initialized
Minibatch loss at step 0: 3.455361
Minibatch accuracy: 0.0%
Valid accuracy: 10.1%
Minibatch loss at step 50: 2.086510
Minibatch accuracy: 31.2%
Valid accuracy: 28.6%
Minibatch loss at step 100: 1.407345
Minibatch accuracy: 37.5%
Valid accuracy: 49.3%
Minibatch loss at step 150: 0.598665
Minibatch accuracy: 87.5%
Valid accuracy: 72.1%
Minibatch loss at step 200: 0.940830
Minibatch accuracy: 68.8%
Valid accuracy: 75.4%
Minibatch loss at step 250: 1.023241
Minibatch accuracy: 68.8%
Valid accuracy: 76.8%
Minibatch loss at step 300: 0.376925
Minibatch accuracy: 87.5%
Valid accuracy: 79.5%
Minibatch loss at step 350: 0.524102
Minibatch accuracy: 93.8%
Valid accuracy: 77.4%
Minibatch loss at step 400: 0.178002
Minibatch accuracy: 100.0%
Valid accuracy: 80.5%
Minibatch loss at step 450: 0.768198
Minibatch accuracy: 81.2%
Valid accuracy: 78.6%
Minibatch loss at step 500: 0.828625
Minibatch accuracy: 81.2%
Valid accuracy: 80.8%
Minibatch loss at step 550: 0.786381
Minibatch accuracy:

**avg_pool almost have the same performance as max_pool**

### Try inception

Naive inception

<img src="./naive_inception.png">

In [132]:
batch_size = 16
patch_size_1 = 1
patch_size_3 = 3
patch_size_5 = 5
depth = 32 
num_hidden = 1024
strides = 2
ksize = 3
dropout_rate = 0.5
shape_size = image_size // 4 * image_size // 4 * depth * 4
batch_size_2 = test_dataset.shape[0] // 10

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.placeholder(
        tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))
    tf_test_dataset = tf.placeholder(
        tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))

    # Variables and model.
    ## layer1:
    layer1_1_weights = tf.Variable(tf.truncated_normal(
        [patch_size_1, patch_size_1, num_channels, depth], stddev=0.1))
    layer1_1_biases = tf.Variable(tf.zeros([depth]))
    

    layer1_3_weights = tf.Variable(tf.truncated_normal(
        [patch_size_3, patch_size_3, num_channels, depth], stddev=0.1))
    layer1_3_biases = tf.Variable(tf.zeros([depth]))                                  
    

    layer1_5_weights = tf.Variable(tf.truncated_normal(
            [patch_size_5, patch_size_5, num_channels, depth], stddev=0.1))
    layer1_5_biases = tf.Variable(tf.zeros([depth]))
    

    layer1_pool_weights = tf.Variable(tf.truncated_normal(
        [patch_size_3, patch_size_3, num_channels, depth], stddev=0.1))
    layer1_pool_biases = tf.Variable(tf.zeros([depth]))
    

    ## layer2:
    layer2_1_weights = tf.Variable(tf.truncated_normal(
        [patch_size_1, patch_size_1, depth, depth], stddev=0.1))
    layer2_1_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    

    layer2_3_weights = tf.Variable(tf.truncated_normal(
        [patch_size_3, patch_size_3, depth, depth], stddev=0.1))
    layer2_3_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer2_5_weights = tf.Variable(tf.truncated_normal(
        [patch_size_5, patch_size_5, depth, depth], stddev=0.1))
    layer2_5_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    

    layer2_pool_weights = tf.Variable(tf.truncated_normal(
        [patch_size_5, patch_size_5, depth, depth], stddev=0.1))
    layer2_pool_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    

    ## layer3
    layer3_weights = tf.Variable(tf.truncated_normal([shape_size, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    def model_naive_inception(data):
        conv_1_1 = tf.nn.conv2d(data, layer1_1_weights, [1, strides, strides, 1], padding='SAME')
        hidden_1_1 = tf.nn.relu(conv_1_1 + layer1_1_biases)

        conv_1_3 = tf.nn.conv2d(data, layer1_3_weights, [1, strides, strides, 1], padding='SAME')
        hidden_1_3 = tf.nn.relu(conv_1_3 + layer1_3_biases)

        conv_1_5 = tf.nn.conv2d(data, layer1_5_weights, [1, strides, strides, 1], padding='SAME')
        hidden_1_5 = tf.nn.relu(conv_1_5 + layer1_5_biases) 

        conv_1_pool = tf.nn.conv2d(data, layer1_pool_weights, [1, strides, strides, 1], padding='SAME')
        hidden_1_pool = tf.nn.relu(conv_1_pool + layer1_pool_biases) 
        hidden_1_pool = tf.nn.max_pool(hidden_1_pool, [1, ksize, ksize, 1], [1, 1, 1, 1,], padding='SAME') 

        conv_2_1 = tf.nn.conv2d(hidden_1_1, layer2_1_weights, [1, strides, strides, 1], padding='SAME')
        hidden_2_1 = tf.nn.relu(conv_2_1 + layer2_1_biases)

        conv_2_3 = tf.nn.conv2d(hidden_1_3, layer2_3_weights, [1, strides, strides, 1], padding='SAME')
        hidden_2_3 = tf.nn.relu(conv_2_3 + layer2_3_biases) 

        conv_2_5 = tf.nn.conv2d(hidden_1_5, layer2_5_weights, [1, strides, strides, 1], padding='SAME')
        hidden_2_5 = tf.nn.relu(conv_2_5 + layer2_5_biases) 

        conv_2_pool = tf.nn.conv2d(hidden_1_pool, layer2_pool_weights, [1, strides, strides, 1], padding='SAME')
        hidden_2_pool = tf.nn.relu(conv_2_pool + layer2_pool_biases) 
        hidden_2_pool = tf.nn.max_pool(hidden_2_pool, [1, ksize, ksize, 1], [1, 1, 1, 1], padding='SAME') 

        shape_1 = hidden_2_1.get_shape().as_list()
        reshape_1 = tf.reshape(hidden_2_1, [shape_1[0], shape_1[1] * shape_1[2] * shape_1[3]])

        shape_3 = hidden_2_3.get_shape().as_list()
        reshape_3 = tf.reshape(hidden_2_3, [shape_3[0], shape_3[1] * shape_3[2] * shape_3[3]])

        shape_5 = hidden_2_5.get_shape().as_list()
        reshape_5 = tf.reshape(hidden_2_5, [shape_5[0], shape_5[1] * shape_5[2] * shape_5[3]])

        shape_pool = hidden_2_pool.get_shape().as_list()
        reshape_pool = tf.reshape(hidden_2_pool, [shape_pool[0], shape_pool[1] * shape_pool[2] * shape_pool[3]])

        reshape = tf.concat(1, [reshape_1,reshape_3,reshape_5, reshape_pool]) 
        hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),dropout_rate) 

        return tf.matmul(hidden, layer4_weights) + layer4_biases
    

    # Training computation.
    logits = model_naive_inception(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)


    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model_naive_inception(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model_naive_inception(tf_test_dataset))

In [135]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    batch_data, batch_labels = batch(train_dataset, train_labels,batch_size)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      a = []  
      for step in range(10):
        valid_batch_data, valid_batch_labels = batch(valid_dataset, valid_labels, batch_size_2)
        feed_dict = {tf_valid_dataset : valid_batch_data}
        valid_predictions= session.run(valid_prediction, feed_dict=feed_dict)
        a.append(accuracy(valid_predictions, valid_batch_labels))
      print('Valid accuracy: %.1f%%' % (sum(a)/len(a)))
  a = []  
  for step in range(10):
    test_batch_data, test_batch_labels = batch(test_dataset, test_labels, batch_size_2)
    feed_dict = {tf_test_dataset : test_batch_data}
    test_predictions= session.run(test_prediction, feed_dict=feed_dict)
    a.append(accuracy(test_predictions, test_batch_labels))
  print('Test accuracy: %.1f%%' % (sum(a)/len(a)))

Initialized
Minibatch loss at step 0: 28.039040
Minibatch accuracy: 0.0%
Valid accuracy: 10.3%
Minibatch loss at step 50: 2.252144
Minibatch accuracy: 12.5%
Valid accuracy: 21.7%
Minibatch loss at step 100: 2.321068
Minibatch accuracy: 18.8%
Valid accuracy: 47.0%
Minibatch loss at step 150: 0.747953
Minibatch accuracy: 75.0%
Valid accuracy: 63.8%
Minibatch loss at step 200: 1.233307
Minibatch accuracy: 75.0%
Valid accuracy: 67.1%
Minibatch loss at step 250: 1.395008
Minibatch accuracy: 56.2%
Valid accuracy: 69.8%
Minibatch loss at step 300: 0.535615
Minibatch accuracy: 81.2%
Valid accuracy: 72.4%
Minibatch loss at step 350: 0.407672
Minibatch accuracy: 93.8%
Valid accuracy: 72.8%
Minibatch loss at step 400: 0.438156
Minibatch accuracy: 81.2%
Valid accuracy: 73.1%
Minibatch loss at step 450: 0.864623
Minibatch accuracy: 81.2%
Valid accuracy: 73.5%
Minibatch loss at step 500: 1.012808
Minibatch accuracy: 75.0%
Valid accuracy: 76.2%
Minibatch loss at step 550: 0.779269
Minibatch accuracy:

**The inception works as well as CNN + max_pool**

## Add L2_penalty

In [136]:
l2_weights = 1e-3
batch_size = 16
patch_size_1 = 1
patch_size_3 = 3
patch_size_5 = 5
depth = 32 
num_hidden = 1024
strides = 2
ksize = 3
dropout_rate = 0.5
shape_size = image_size // 4 * image_size // 4 * depth * 4
batch_size_2 = test_dataset.shape[0] // 10

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.placeholder(
        tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))
    tf_test_dataset = tf.placeholder(
        tf.float32, shape=(batch_size_2, image_size, image_size, num_channels))

    # Variables and model.
    ## layer1:
    layer1_1_weights = tf.Variable(tf.truncated_normal(
        [patch_size_1, patch_size_1, num_channels, depth], stddev=0.1))
    layer1_1_biases = tf.Variable(tf.zeros([depth]))
    

    layer1_3_weights = tf.Variable(tf.truncated_normal(
        [patch_size_3, patch_size_3, num_channels, depth], stddev=0.1))
    layer1_3_biases = tf.Variable(tf.zeros([depth]))                                  
    

    layer1_5_weights = tf.Variable(tf.truncated_normal(
            [patch_size_5, patch_size_5, num_channels, depth], stddev=0.1))
    layer1_5_biases = tf.Variable(tf.zeros([depth]))
    

    layer1_pool_weights = tf.Variable(tf.truncated_normal(
        [patch_size_3, patch_size_3, num_channels, depth], stddev=0.1))
    layer1_pool_biases = tf.Variable(tf.zeros([depth]))
    

    ## layer2:
    layer2_1_weights = tf.Variable(tf.truncated_normal(
        [patch_size_1, patch_size_1, depth, depth], stddev=0.1))
    layer2_1_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    

    layer2_3_weights = tf.Variable(tf.truncated_normal(
        [patch_size_3, patch_size_3, depth, depth], stddev=0.1))
    layer2_3_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer2_5_weights = tf.Variable(tf.truncated_normal(
        [patch_size_5, patch_size_5, depth, depth], stddev=0.1))
    layer2_5_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    

    layer2_pool_weights = tf.Variable(tf.truncated_normal(
        [patch_size_5, patch_size_5, depth, depth], stddev=0.1))
    layer2_pool_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    

    ## layer3
    layer3_weights = tf.Variable(tf.truncated_normal([shape_size, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

    layer4_weights = tf.Variable(tf.truncated_normal(
        [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    def model_naive_inception(data):
        conv_1_1 = tf.nn.conv2d(data, layer1_1_weights, [1, strides, strides, 1], padding='SAME')
        hidden_1_1 = tf.nn.relu(conv_1_1 + layer1_1_biases)

        conv_1_3 = tf.nn.conv2d(data, layer1_3_weights, [1, strides, strides, 1], padding='SAME')
        hidden_1_3 = tf.nn.relu(conv_1_3 + layer1_3_biases)

        conv_1_5 = tf.nn.conv2d(data, layer1_5_weights, [1, strides, strides, 1], padding='SAME')
        hidden_1_5 = tf.nn.relu(conv_1_5 + layer1_5_biases) 

        conv_1_pool = tf.nn.conv2d(data, layer1_pool_weights, [1, strides, strides, 1], padding='SAME')
        hidden_1_pool = tf.nn.relu(conv_1_pool + layer1_pool_biases) 
        hidden_1_pool = tf.nn.max_pool(hidden_1_pool, [1, ksize, ksize, 1], [1, 1, 1, 1,], padding='SAME') 

        conv_2_1 = tf.nn.conv2d(hidden_1_1, layer2_1_weights, [1, strides, strides, 1], padding='SAME')
        hidden_2_1 = tf.nn.relu(conv_2_1 + layer2_1_biases)

        conv_2_3 = tf.nn.conv2d(hidden_1_3, layer2_3_weights, [1, strides, strides, 1], padding='SAME')
        hidden_2_3 = tf.nn.relu(conv_2_3 + layer2_3_biases) 

        conv_2_5 = tf.nn.conv2d(hidden_1_5, layer2_5_weights, [1, strides, strides, 1], padding='SAME')
        hidden_2_5 = tf.nn.relu(conv_2_5 + layer2_5_biases) 

        conv_2_pool = tf.nn.conv2d(hidden_1_pool, layer2_pool_weights, [1, strides, strides, 1], padding='SAME')
        hidden_2_pool = tf.nn.relu(conv_2_pool + layer2_pool_biases) 
        hidden_2_pool = tf.nn.max_pool(hidden_2_pool, [1, ksize, ksize, 1], [1, 1, 1, 1], padding='SAME') 

        shape_1 = hidden_2_1.get_shape().as_list()
        reshape_1 = tf.reshape(hidden_2_1, [shape_1[0], shape_1[1] * shape_1[2] * shape_1[3]])

        shape_3 = hidden_2_3.get_shape().as_list()
        reshape_3 = tf.reshape(hidden_2_3, [shape_3[0], shape_3[1] * shape_3[2] * shape_3[3]])

        shape_5 = hidden_2_5.get_shape().as_list()
        reshape_5 = tf.reshape(hidden_2_5, [shape_5[0], shape_5[1] * shape_5[2] * shape_5[3]])

        shape_pool = hidden_2_pool.get_shape().as_list()
        reshape_pool = tf.reshape(hidden_2_pool, [shape_pool[0], shape_pool[1] * shape_pool[2] * shape_pool[3]])

        reshape = tf.concat(1, [reshape_1,reshape_3,reshape_5, reshape_pool]) 
        hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases),dropout_rate) 

        return tf.matmul(hidden, layer4_weights) + layer4_biases
    

    # Training computation.
    logits = model_naive_inception(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    layer1_l2_loss = tf.nn.l2_loss(layer1_1_weights) + tf.nn.l2_loss(layer1_3_weights) + tf.nn.l2_loss(layer1_5_weights) + tf.nn.l2_loss(layer1_pool_weights)
    layer2_l2_loss = tf.nn.l2_loss(layer2_1_weights) + tf.nn.l2_loss(layer2_3_weights) + tf.nn.l2_loss(layer2_5_weights) + tf.nn.l2_loss(layer2_pool_weights)
    l2_loss = layer1_l2_loss + layer2_l2_loss + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer4_weights)
    sum_loss = loss + l2_loss*l2_weights

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(sum_loss)


    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model_naive_inception(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model_naive_inception(tf_test_dataset))

In [137]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    batch_data, batch_labels = batch(train_dataset, train_labels,batch_size)
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      a = []  
      for step in range(10):
        valid_batch_data, valid_batch_labels = batch(valid_dataset, valid_labels, batch_size_2)
        feed_dict = {tf_valid_dataset : valid_batch_data}
        valid_predictions= session.run(valid_prediction, feed_dict=feed_dict)
        a.append(accuracy(valid_predictions, valid_batch_labels))
      print('Valid accuracy: %.1f%%' % (sum(a)/len(a)))
  a = []  
  for step in range(10):
    test_batch_data, test_batch_labels = batch(test_dataset, test_labels, batch_size_2)
    feed_dict = {tf_test_dataset : test_batch_data}
    test_predictions= session.run(test_prediction, feed_dict=feed_dict)
    a.append(accuracy(test_predictions, test_batch_labels))
  print('Test accuracy: %.1f%%' % (sum(a)/len(a)))

Initialized
Minibatch loss at step 0: 29.662697
Minibatch accuracy: 6.2%
Valid accuracy: 10.2%
Minibatch loss at step 50: 2.165970
Minibatch accuracy: 31.2%
Valid accuracy: 21.4%
Minibatch loss at step 100: 2.644716
Minibatch accuracy: 31.2%
Valid accuracy: 33.2%
Minibatch loss at step 150: 1.385363
Minibatch accuracy: 56.2%
Valid accuracy: 51.8%
Minibatch loss at step 200: 1.227969
Minibatch accuracy: 68.8%
Valid accuracy: 63.7%
Minibatch loss at step 250: 1.320062
Minibatch accuracy: 56.2%
Valid accuracy: 64.8%
Minibatch loss at step 300: 0.561086
Minibatch accuracy: 81.2%
Valid accuracy: 70.7%
Minibatch loss at step 350: 0.758780
Minibatch accuracy: 87.5%
Valid accuracy: 72.0%
Minibatch loss at step 400: 0.276872
Minibatch accuracy: 93.8%
Valid accuracy: 68.9%
Minibatch loss at step 450: 0.872654
Minibatch accuracy: 81.2%
Valid accuracy: 71.9%
Minibatch loss at step 500: 0.868940
Minibatch accuracy: 81.2%
Valid accuracy: 73.2%
Minibatch loss at step 550: 1.016740
Minibatch accuracy:

**Add l2_penalty make no difference**

## Try complex inception

<img src="./inception.png">